In [2]:
!pip install pulp

In [208]:
import pulp as op
import numpy as np

In [216]:
p = np.array([1,2,3]) #precios de las variedades.
#B = np.array([[0,0,0],[0,0,0]]) #matriz binaria de camas por variedad sembrada.
w = np.array([150,250,350,25,25,25,25,1000]) #capacidad de siembra de los bancos.
v = np.array([173,350,100]) # cantidades de variedades disponibles para sembrar.
n = v.shape[0] #Cantidad de variedades.
m = w.shape[0] #Cantidad de camas.
L=np.array([[0,0,0],[0,0,0]]) #Restricciones de NO compatibilidad por variedad.

In [217]:
concat = lambda t,e : str(t)+str(e)
beds = [concat(xt,xe) for xt,xe in zip(np.repeat(['Bed'],m*n),np.repeat(np.arange(1,m+1),n))]
varieties = [concat(xt,xe) for xt,xe in zip(np.repeat(['_Var'],m*n),np.tile(np.arange(1,n+1),m))]
bedsXvars = [concat(xt,xe) for xt,xe in zip(beds,varieties)]
c=dict(zip(bedsXvars, np.tile(p,m)*np.repeat(w,n)))
A=dict(zip(bedsXvars,np.repeat(w,n)))

In [218]:
prob = op.LpProblem("Plannting_plan",op.LpMaximize)
beds = op.LpVariable.dicts("BedsVar",bedsXvars,lowBound=0,upBound=1,cat='Integer')
prob += op.lpSum([c[i]*beds[i] for i in bedsXvars]) # Función de Costo, Precio de la variedad * X

Restricción que representa la capacidad de los bancos por variedad

In [219]:
for j in np.arange(0,n):
    rTemp = np.zeros(m*n)
    for h in np.arange(0,m):
        p1 = np.zeros(j) #Cantidad de ceros adelante segun el indice de la cama
        p2 = [w[h]] #Capacidad del banco
        p3 = np.zeros(n-(j+1)) #
        rTemp[h*n:(h*n)+3]=np.concatenate((p1,p2,p3))
    r = dict(zip(bedsXvars,rTemp))
    prob += op.lpSum([r[i]*beds[i] for i in bedsXvars])<=v[j]*1.1 #Restricción del área de la siembra.

Restricción que representa que solo pueda sembrarse una variedad por cama

In [220]:
for j in np.arange(0,m): #Se crea un restricción por cama.
    p1 = np.zeros(j*n) #Cantidad de ceros adelante segun el indice de la cama
    p2 = np.ones(n) #Cantidad de unos que representan la restriccion
    p3 = np.zeros(((m*n)-((j*n)+n))) #cantidad de ceros al final
    r = dict(zip(bedsXvars,np.concatenate((p1,p2,p3))))
    prob += op.lpSum([r[i]*beds[i] for i in bedsXvars]) <= 1 #Restricción de máximo una variedad por cama.

In [221]:
#for i in np.arange(0,L.shape[0]): #Se crea un restricción por cama.
#    varCom = dict(zip(bedsXvars,np.tile(L[i],m)))
#    prob += op.lpSum([varCom[i]*beds[i] for i in bedsXvars]) == 0 #Restricción de compatibilidad de flor por BLOQUE.

In [222]:
prob.solve()
print("Estado:",op.LpStatus[prob.status])
print("Objetivo:",op.value(prob.objective))
print("Vaiables:")
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Estado: Optimal
Objetivo: 1150.0
Vaiables:
BedsVar_Bed1_Var1 = 1.0
BedsVar_Bed3_Var2 = 1.0
BedsVar_Bed4_Var3 = 1.0
BedsVar_Bed5_Var3 = 1.0
BedsVar_Bed6_Var3 = 1.0
BedsVar_Bed7_Var3 = 1.0


In [190]:
prob

Plannting_plan:
MAXIMIZE
150*BedsVar_Bed1_Var1 + 300*BedsVar_Bed1_Var2 + 450*BedsVar_Bed1_Var3 + 250*BedsVar_Bed2_Var1 + 500*BedsVar_Bed2_Var2 + 750*BedsVar_Bed2_Var3 + 350*BedsVar_Bed3_Var1 + 700*BedsVar_Bed3_Var2 + 1050*BedsVar_Bed3_Var3 + 25*BedsVar_Bed4_Var1 + 50*BedsVar_Bed4_Var2 + 75*BedsVar_Bed4_Var3 + 25*BedsVar_Bed5_Var1 + 50*BedsVar_Bed5_Var2 + 75*BedsVar_Bed5_Var3 + 25*BedsVar_Bed6_Var1 + 50*BedsVar_Bed6_Var2 + 75*BedsVar_Bed6_Var3 + 25*BedsVar_Bed7_Var1 + 50*BedsVar_Bed7_Var2 + 75*BedsVar_Bed7_Var3 + 1000*BedsVar_Bed8_Var1 + 2000*BedsVar_Bed8_Var2 + 3000*BedsVar_Bed8_Var3 + 0
SUBJECT TO
_C1: 150 BedsVar_Bed1_Var1 + 250 BedsVar_Bed2_Var1 + 350 BedsVar_Bed3_Var1
 + 25 BedsVar_Bed4_Var1 + 25 BedsVar_Bed5_Var1 + 25 BedsVar_Bed6_Var1
 + 25 BedsVar_Bed7_Var1 + 1000 BedsVar_Bed8_Var1 <= 100

_C2: 150 BedsVar_Bed1_Var2 + 250 BedsVar_Bed2_Var2 + 350 BedsVar_Bed3_Var2
 + 25 BedsVar_Bed4_Var2 + 25 BedsVar_Bed5_Var2 + 25 BedsVar_Bed6_Var2
 + 25 BedsVar_Bed7_Var2 + 1000 BedsVar_Bed8_Var